In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import small_rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from general_task_network import student_network
from utils import dotdict
from keras import regularizers

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

core_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 300,
            "learning_rate" : 0.001,
            "residual_layers" : 3
})

value_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 300,
            "learning_rate" : 0.001,
            "post_core_residual_layers" : 2,
            "reward_fork_layers": 1,
            "value_fork_layers": 1
})

state_network_params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.05,
            "residual_units" : 300,
            "learning_rate" : 0.0001,
            "post_core_residual_layers" : 2
})

params = dotdict({
    "core_params":core_params,
    "value_network_params": value_network_params,
    "state_network_params": state_network_params,
    "state_size":task.state_size,
    "action_codes": task.n_actions
})

my_student_network = student_network.create(params)
#my_student_network = student_network.load_value_network("models/trained.h5")
student_template = student(task, 30,my_student_network,0.95)
t = teacher(setup, lambda n : 1+rng.poisson(lam=3,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10, buffer_size = lambda n : 2048)


In [2]:
my_student_network.save("models/combined/model_1800_03_01")

INFO:tensorflow:Assets written to: models/combined/model_1800_03_01_value_network.ckpt\assets


INFO:tensorflow:Assets written to: models/combined/model_1800_03_01_value_network.ckpt\assets


INFO:tensorflow:Assets written to: models/combined/model_1800_03_01_state_network.ckpt\assets


INFO:tensorflow:Assets written to: models/combined/model_1800_03_01_state_network.ckpt\assets


Network saved under models/combined/model_1800_03_01.


In [3]:
loaded = student_network.load("models/combined/model_1800_03_01")

In [6]:
c.max_steps=15
t.step_dist = lambda n : 8*np.ones(n,dtype=int)

In [ ]:

for i in range(10):
    print(f"Round {i}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)
    #if i % 10 == 9:
    #    my_student_network.save("models/trained.h5")

In [13]:
c.buffer_size = lambda n : 8192

In [4]:
n=8192
rng = np.random.default_rng(seed=0)
states = np.arange(24)[None,:].repeat(n,axis=0)

for k in range(10):
    print(f"Round {k}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)

    for i in range(30):
        _, states = rubiks.task_action(states,rng.choice(a=12,size=n))

    actions = rng.choice(12,size=n)
    _, next_states = rubiks.task_action(states,actions)

    actions = (np.arange(12) == actions[...,None]).astype(float)
    colors = rubiks.make_neural_input(states)
    next_colors = rubiks.make_neural_input(next_states)
    my_student_network.fit_state(colors,actions,next_colors,epochs = 5)

Round 0.
Before step 1, 94 out of 100 remain open.


ValueError: Input 0 of layer "model" is incompatible with the layer: expected shape=(None, 324), found shape=(94, 144)

In [ ]:
states = np.arange(24)[None,:]
actions = np.array([0])
prediction = my_student_network.predict_state(states,actions)
prediction.reshape(1,24,6), prediction.reshape(1,24,6).argmax(axis=2), rubiks.start_coloring[rubiks.task_action(states,actions)[1][0]]